# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [1]:
import requests
import json
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
HABR_URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']
DATA_FRAME = {"date": [], "title": [], "link": [], "text": []}

In [3]:
req = requests.get(HABR_URL)
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [4]:
def get_post_text(link):
    req = requests.get(link)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text, 'html.parser')
    text = soup.find('div', {'id': 'post-content-body'}).get_text()
    return text

In [5]:
def filter_posts(post):
    full_link = post.find('a', class_='post__title_link')
    link = full_link.get('href')
    title = full_link.get_text()
    date = post.find('span', class_='post__time')
    text = get_post_text(link)
    
    if any([kwd in text.lower() for kwd in KEYWORDS]):
        DATA_FRAME["link"].append(link)
        DATA_FRAME["title"].append(title)
        DATA_FRAME["date"].append(date)
        DATA_FRAME["text"].append(text)

In [ ]:
for post in posts:
    filter_posts(post)
        
post_df = pd.DataFrame(DATA_FRAME)

post_df

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [ ]:
EMAIL = ["xxx@x.ru", "yyy@y.com"]
url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"

In [ ]:
headers = {'Accept': '*/*', 
        'Accept-Encoding':'gzip, deflate, br',
        'Content-Length':'31',
        'Content-Type':'application/json',
        'Host':'identityprotection.avast.com',
        'User-Agent':'PostmanRuntime/7.26.10',
        'Connection':'keep-alive',
        'Vaar-Header-App-Build-Version':'1.0.0',
        'Vaar-Header-App-Product-Name':'hackcheck-web-avast',
        'Vaar-Version':'0'
        }

In [ ]:
res_df = pd.DataFrame(columns=('email', 'leak_date', 'leak_source', 'description'))

for email in EMAIL:
    payload = {"emailAddresses":[email]}
    res = requests.post(url, headers=headers, data=json.dumps(payload))
    try:
        result = pd.DataFrame(res.json()['breaches'])     
        if len(result.index)==0:
            print('Почта '+email+' не взломана')

        for item in result:
            df = pd.DataFrame(
                {"email": [email],
                "leak_date": [result[item]['publishDate']],
                "leak_source": [result[item]['site']],
                "description": [result['description']]})
            res_df = res_df.append(df)
        
    except:
        print('Для почты '+email+' cайт выдал Captcha.')
        
res_df

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`